In [1]:
#DATA ENGINEERING
import csv
data={}
with open('C:\\Users\\Hp\\distracted-driver-detection\\driver_imgs_list.csv') as f:
    reader=csv.reader(f)
    next(reader)#for skipping first row
    for row in reader:
        key=row[1].lower()
        if key in data:
            data[key].append(row[2])
        else:
            data[key]=[row[2]]

In [2]:
import os

In [3]:
data

{'normal driving': ['img_44733.jpg',
  'img_72999.jpg',
  'img_25094.jpg',
  'img_69092.jpg',
  'img_92629.jpg',
  'img_3370.jpg',
  'img_67639.jpg',
  'img_58560.jpg',
  'img_35779.jpg',
  'img_10012.jpg',
  'img_70354.jpg',
  'img_54617.jpg',
  'img_26684.jpg',
  'img_82409.jpg',
  'img_5585.jpg',
  'img_19029.jpg',
  'img_40930.jpg',
  'img_31547.jpg',
  'img_87373.jpg',
  'img_10627.jpg',
  'img_66355.jpg',
  'img_97714.jpg',
  'img_37344.jpg',
  'img_54848.jpg',
  'img_59964.jpg',
  'img_70675.jpg',
  'img_43317.jpg',
  'img_13585.jpg',
  'img_63064.jpg',
  'img_91058.jpg',
  'img_79799.jpg',
  'img_18198.jpg',
  'img_55108.jpg',
  'img_87789.jpg',
  'img_11582.jpg',
  'img_24152.jpg',
  'img_28404.jpg',
  'img_8868.jpg',
  'img_14949.jpg',
  'img_41276.jpg',
  'img_70611.jpg',
  'img_42895.jpg',
  'img_87995.jpg',
  'img_40990.jpg',
  'img_68866.jpg',
  'img_71265.jpg',
  'img_90102.jpg',
  'img_30954.jpg',
  'img_3570.jpg',
  'img_59000.jpg',
  'img_48323.jpg',
  'img_13073.jpg'

In [4]:
distracted_list=list(data.keys())
distracted_list

['normal driving',
 'texting - right',
 'talking on the phone - right',
 'texting - left',
 'talking on the phone - left',
 'operating the radio',
 'drinking',
 'reaching behind',
 'hair and makeup',
 'talking to passenger']

In [5]:
os.mkdir('distracted_data')
os.mkdir('distracted_data/training')
os.mkdir('distracted_data/testing')

In [6]:
for distraction in distracted_list:
    os.mkdir(os.path.join('distracted_data/training/',distraction))
    os.mkdir(os.path.join('distracted_data/testing/',distraction))

In [7]:
def replace_folder_name(directory_path, old_folder_name, new_folder_name):
    # Get the list of all items in the directory
    items = os.listdir(directory_path)

    # Iterate through each item
    for item in items:
        item_path = os.path.join(directory_path, item)

        # Check if the item is a directory and matches the old folder name
        if os.path.isdir(item_path) and item == old_folder_name:
            # Create the new path with the new folder name
            new_item_path = os.path.join(directory_path, new_folder_name)

            # Rename the folder
            os.rename(item_path, new_item_path)

            print(f"Renamed folder: {item_path} to {new_item_path}")

directory_path = "C:\\Users\\Hp\\distracted-driver-detection\\imgs\\train"

replace_folder_name(directory_path, "c0", "normal driving")


In [8]:
replace_folder_name(directory_path, "c1", "texting - right")
replace_folder_name(directory_path, "c2", "talking on the phone - right")
replace_folder_name(directory_path, "c3", "texting - left")
replace_folder_name(directory_path, "c4", "talking on the phone - left")
replace_folder_name(directory_path, "c5", "operating the radio")
replace_folder_name(directory_path, "c6", "drinking")
replace_folder_name(directory_path, "c7", "reaching behind")
replace_folder_name(directory_path, "c8", "hair and makeup")
replace_folder_name(directory_path, "c9", "talking to passenger")

In [10]:
from shutil import copyfile
split_size=0.8

for distraction,images in data.items():
    train_size=int(split_size*len(images))
    train_images=images[:train_size]
    test_images=images[train_size:]
    for image in train_images:
        source=os.path.join('C:\\Users\\Hp\\distracted-driver-detection\\imgs\\train\\',distraction,image)
        dest=os.path.join('C:\\Users\\Hp\\distracted_data\\training\\',distraction,image)
        copyfile(source,dest)
    for image in test_images:
        source=os.path.join('C:\\Users\\Hp\\distracted-driver-detection\\imgs\\train\\',distraction,image)
        dest=os.path.join('C:\\Users\\Hp\\distracted_data\\testing\\',distraction,image)
        copyfile(source,dest)

In [11]:
#FEATURE ENGINEERING
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [12]:
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2), 
    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2), 
    Flatten(),
    Dense(512,activation='relu'),
    Dense(10,activation='softmax')
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 64)        0

In [13]:
#TRAIN/TEST SPLIT
train_dir='C:\\Users\\Hp\\distracted_data\\training'
test_dir='C:\\Users\\Hp\\distracted_data\\testing'

#training data
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(100,100),class_mode='categorical',batch_size=128)

#validation data
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(test_dir,target_size=(100,100),class_mode='categorical',batch_size=128)

Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [14]:
#HYPER PARAMETER TUNING (WE HAVE USED EARLY STOP AS HYPERPARAMETER)
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [15]:
#MODEL SELECTION AND DEPLOYMENT
model.fit(train_generator,epochs=10,validation_data=test_generator,callbacks=[es])
#PERFORMANCE MATRIX (BESIDE EVERY EPOCH WE HAVE SHOWN COMPONENTS OF PERFORMANCE MATRIX)

Epoch 1/10
141/141 [==============================] - 208s 1s/step - loss: 0.7919 - acc: 0.7439 - val_loss: 5.7272 - val_acc: 0.1842
Epoch 2/10
141/141 [==============================] - 101s 716ms/step - loss: 0.0763 - acc: 0.9812 - val_loss: 6.7962 - val_acc: 0.2214
Epoch 3/10
141/141 [==============================] - 101s 716ms/step - loss: 0.0336 - acc: 0.9904 - val_loss: 7.0356 - val_acc: 0.2140
Epoch 4/10
141/141 [==============================] - 99s 706ms/step - loss: 0.0200 - acc: 0.9941 - val_loss: 7.4880 - val_acc: 0.2256


In [16]:
test2_datagen=ImageDataGenerator(rescale=1.0/255)
test2_generator=test2_datagen.flow_from_directory("C:\\Users\\Hp\\distracted-driver-detection\\imgs\\test",target_size=(100,100),class_mode=None,batch_size=128)

Found 0 images belonging to 0 classes.


In [17]:
import pandas as pd

test2_directory = "C:\\Users\\Hp\\distracted-driver-detection\\imgs\\test"
img_height, img_width = 100, 100  # Adjust based on your model's input size
batch_size = 128  # Adjust based on your preferences

# Get a list of file paths
file_paths = [os.path.join(test2_directory, filename) for filename in os.listdir(test2_directory) if os.path.isfile(os.path.join(test2_directory, filename))]

# Create a DataFrame with the file paths
df = pd.DataFrame({'filename': file_paths})

# Create an ImageDataGenerator for testing
test2_datagen = ImageDataGenerator(rescale=1./255)

# Create a generator for testing
test2_generator = test2_datagen.flow_from_dataframe(
    dataframe=df,
    x_col='filename',
    y_col=None,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)



Found 79726 validated image filenames.


In [18]:
arr=model.predict(test2_generator)

623/623 [==============================] - 662s 1s/step


In [21]:
#above array predicts how much percentage each depicts for that category therefore the maximum value of the array will represent the category for that particular value

In [19]:
arr

array([[7.2884712e-15, 3.3035220e-13, 2.7390074e-14, ..., 2.2984974e-09,
        4.9142118e-10, 3.7241334e-12],
       [4.9416309e-07, 2.5046549e-07, 6.8012127e-07, ..., 5.5982428e-06,
        1.1112772e-01, 2.0269225e-08],
       [3.4000212e-04, 2.4501992e-02, 1.8301365e-01, ..., 2.3332360e-01,
        1.7004226e-04, 3.5445089e-03],
       ...,
       [8.0075843e-05, 1.4356739e-05, 1.7215134e-06, ..., 1.5285687e-03,
        2.6125453e-13, 9.9820745e-01],
       [6.3097805e-01, 2.6519693e-04, 5.3822912e-08, ..., 1.7004007e-06,
        3.9932289e-09, 5.5431487e-06],
       [2.9568856e-09, 9.4359034e-01, 2.6478721e-09, ..., 4.9946852e-05,
        2.0884539e-07, 1.6792263e-05]], dtype=float32)

In [20]:
ARR=["drinking","hair and makeup","normal driving","operating the radio","reaching behind","talking on the phone - left","talking on the phone - right","talking to passenger","texting - left","texting - right"]

In [26]:
#COMPARATIVE ANALYSIS
print("PREDICTIONS FOR TEST IMAGES:",end="\n")
x=1
for i in arr:
    maxi=0
    ind=-1
    print("Test image ",x,":",end=" ")
    x+=1
    for j in range(i.size):
        if(i[j]>maxi):
            maxi=i[j]
            ind=j
    print(ARR[ind],end="\n")

PREDICTIONS FOR TEST IMAGES:
Test image  1 : operating the radio
Test image  2 : operating the radio
Test image  3 : talking on the phone - right
Test image  4 : texting - right
Test image  5 : talking on the phone - left
Test image  6 : texting - right
Test image  7 : texting - right
Test image  8 : texting - right
Test image  9 : drinking
Test image  10 : reaching behind
Test image  11 : hair and makeup
Test image  12 : hair and makeup
Test image  13 : texting - right
Test image  14 : normal driving
Test image  15 : talking on the phone - right
Test image  16 : texting - right
Test image  17 : operating the radio
Test image  18 : texting - left
Test image  19 : talking to passenger
Test image  20 : talking to passenger
Test image  21 : talking on the phone - right
Test image  22 : talking on the phone - right
Test image  23 : talking to passenger
Test image  24 : reaching behind
Test image  25 : texting - left
Test image  26 : talking to passenger
Test image  27 : talking on the phon

Test image  11939 : talking on the phone - left
Test image  11940 : talking to passenger
Test image  11941 : talking on the phone - left
Test image  11942 : talking on the phone - left
Test image  11943 : operating the radio
Test image  11944 : talking on the phone - left
Test image  11945 : talking to passenger
Test image  11946 : hair and makeup
Test image  11947 : reaching behind
Test image  11948 : talking to passenger
Test image  11949 : texting - left
Test image  11950 : texting - left
Test image  11951 : drinking
Test image  11952 : texting - right
Test image  11953 : talking to passenger
Test image  11954 : talking on the phone - left
Test image  11955 : hair and makeup
Test image  11956 : talking on the phone - left
Test image  11957 : reaching behind
Test image  11958 : texting - right
Test image  11959 : talking to passenger
Test image  11960 : normal driving
Test image  11961 : talking on the phone - left
Test image  11962 : talking to passenger
Test image  11963 : talking 

Test image  21752 : hair and makeup
Test image  21753 : operating the radio
Test image  21754 : texting - left
Test image  21755 : talking on the phone - left
Test image  21756 : reaching behind
Test image  21757 : operating the radio
Test image  21758 : texting - right
Test image  21759 : talking on the phone - left
Test image  21760 : texting - right
Test image  21761 : talking to passenger
Test image  21762 : reaching behind
Test image  21763 : texting - right
Test image  21764 : talking to passenger
Test image  21765 : talking to passenger
Test image  21766 : reaching behind
Test image  21767 : talking on the phone - left
Test image  21768 : drinking
Test image  21769 : texting - right
Test image  21770 : texting - left
Test image  21771 : talking on the phone - left
Test image  21772 : texting - right
Test image  21773 : talking to passenger
Test image  21774 : hair and makeup
Test image  21775 : hair and makeup
Test image  21776 : talking to passenger
Test image  21777 : reaching

Test image  32055 : texting - right
Test image  32056 : talking to passenger
Test image  32057 : hair and makeup
Test image  32058 : texting - left
Test image  32059 : hair and makeup
Test image  32060 : talking on the phone - left
Test image  32061 : texting - right
Test image  32062 : reaching behind
Test image  32063 : hair and makeup
Test image  32064 : hair and makeup
Test image  32065 : hair and makeup
Test image  32066 : texting - left
Test image  32067 : texting - right
Test image  32068 : reaching behind
Test image  32069 : reaching behind
Test image  32070 : texting - right
Test image  32071 : operating the radio
Test image  32072 : normal driving
Test image  32073 : normal driving
Test image  32074 : talking to passenger
Test image  32075 : hair and makeup
Test image  32076 : reaching behind
Test image  32077 : talking to passenger
Test image  32078 : texting - right
Test image  32079 : talking on the phone - right
Test image  32080 : talking on the phone - left
Test image  

Test image  43495 : texting - right
Test image  43496 : reaching behind
Test image  43497 : texting - left
Test image  43498 : texting - right
Test image  43499 : texting - left
Test image  43500 : talking on the phone - left
Test image  43501 : texting - left
Test image  43502 : texting - right
Test image  43503 : operating the radio
Test image  43504 : talking on the phone - right
Test image  43505 : normal driving
Test image  43506 : reaching behind
Test image  43507 : texting - left
Test image  43508 : reaching behind
Test image  43509 : reaching behind
Test image  43510 : texting - right
Test image  43511 : hair and makeup
Test image  43512 : talking to passenger
Test image  43513 : reaching behind
Test image  43514 : reaching behind
Test image  43515 : operating the radio
Test image  43516 : reaching behind
Test image  43517 : hair and makeup
Test image  43518 : drinking
Test image  43519 : talking to passenger
Test image  43520 : talking on the phone - right
Test image  43521 : 

Test image  53665 : drinking
Test image  53666 : hair and makeup
Test image  53667 : talking on the phone - left
Test image  53668 : texting - right
Test image  53669 : normal driving
Test image  53670 : texting - left
Test image  53671 : reaching behind
Test image  53672 : talking to passenger
Test image  53673 : talking on the phone - right
Test image  53674 : talking on the phone - left
Test image  53675 : talking to passenger
Test image  53676 : talking to passenger
Test image  53677 : talking to passenger
Test image  53678 : hair and makeup
Test image  53679 : normal driving
Test image  53680 : reaching behind
Test image  53681 : texting - right
Test image  53682 : texting - right
Test image  53683 : talking on the phone - left
Test image  53684 : talking on the phone - left
Test image  53685 : texting - right
Test image  53686 : talking to passenger
Test image  53687 : talking on the phone - left
Test image  53688 : reaching behind
Test image  53689 : talking on the phone - right

Test image  63796 : operating the radio
Test image  63797 : hair and makeup
Test image  63798 : talking to passenger
Test image  63799 : talking on the phone - left
Test image  63800 : texting - right
Test image  63801 : talking on the phone - left
Test image  63802 : normal driving
Test image  63803 : talking on the phone - right
Test image  63804 : normal driving
Test image  63805 : talking to passenger
Test image  63806 : normal driving
Test image  63807 : talking to passenger
Test image  63808 : hair and makeup
Test image  63809 : texting - right
Test image  63810 : texting - right
Test image  63811 : hair and makeup
Test image  63812 : texting - right
Test image  63813 : normal driving
Test image  63814 : hair and makeup
Test image  63815 : texting - left
Test image  63816 : normal driving
Test image  63817 : texting - right
Test image  63818 : talking to passenger
Test image  63819 : hair and makeup
Test image  63820 : reaching behind
Test image  63821 : operating the radio
Test 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

